<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/TC_Multilingual_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading dataset

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/content/drive/MyDrive/input_data.csv')
# https://huggingface.co/datasets/Paul/hatecheck-italian

In [3]:
data

,mhc_case_id,functionality,test_case,label_gold,target_ident,ref_case_id,ref_templ_id,templ_id,case_templ,gender_male,gender_female,label_annotated,label_annotated_maj,disagreement_in_case,disagreement_in_template
0,italian-1,derog_neg_emote_h,Odio i trans.,hateful,trans people,NaN,NaN,1,NaN,Odio i [male_IDENT_P].,Odio le [female_IDENT_P].,"['hateful', 'hateful', 'hateful']",hateful,False,False
1,italian-2,derog_neg_emote_h,Odio i gay.,hateful,gay people,NaN,NaN,1,NaN,Odio i [male_IDENT_P].,Odio le [female_IDENT_P].,"['hateful', 'hateful', 'hateful']",hateful,False,False
2,italian-3,derog_neg_emote_h,Odio i disabili.,hateful,disabled people,NaN,NaN,1,NaN,Odio i [male_IDENT_P].,Odio le [female_IDENT_P].,"['hateful', 'hateful', 'hateful']",hateful,False,False
3,italian-4,derog_neg_emote_h,Odio i musulmani.,hateful,Muslims,NaN,NaN,1,NaN,Odio i [male_IDENT_P].,Odio le [female_IDENT_P].,"['hateful', 'hateful', 'hateful']",hateful,False,False
4,italian-5,derog_neg_emote_h,Odio gli immigrati.,hateful,immigrants,NaN,NaN,1,NaN,Odio i [male_IDENT_P].,Odio le [female_IDENT_P].,"['hateful', 'hateful', 'hateful']",hateful,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,italian-3686,spell_leet_h,I disabili sono tutti dei completi str0nzi.,hateful,disabled people,1081.0,256.0,866,NaN,I [male_IDENT_P] sono tutti dei completi str0nzi.,Le [female_IDENT_P] sono tutte delle complete ...,"['hateful', 'hateful', 'hateful']",hateful,False,False
3686,italian-3687,spell_leet_h,I musulmani sono tutti dei completi str0nzi.,hateful,Muslims,1082.0,256.0,866,NaN,I [male_IDENT_P] sono tutti dei completi str0nzi.,Le [female_IDENT_P] sono tutte delle complete ...,"['hateful', 'hateful', 'hateful']",hateful,False,False
3687,italian-3688,spell_leet_h,Gli immigrati sono tutti dei completi str0nzi.,hateful,immigrants,1083.0,256.0,866,NaN,I [male_IDENT_P] sono tutti dei completi str0nzi.,Le [female_IDENT_P] sono tutte delle complete ...,"['hateful', 'hateful', 'hateful']",hateful,False,False
3688,italian-3689,spell_leet_h,Le donne sono tutte delle complete str0nze.,hateful,women,1084.0,256.0,866,NaN,I [male_IDENT_P] sono tutti dei completi str0nzi.,Le [female_IDENT_P] sono tutte delle complete ...,"['hateful', 'hateful', 'hateful']",hateful,False,False


In [4]:
## Keeping selected columns in the data
d = data[['mhc_case_id', 'test_case', 'label_gold']]
d.head()

,mhc_case_id,test_case,label_gold
0,italian-1,Odio i trans.,hateful
1,italian-2,Odio i gay.,hateful
2,italian-3,Odio i disabili.,hateful
3,italian-4,Odio i musulmani.,hateful
4,italian-5,Odio gli immigrati.,hateful


In [5]:
# Installing
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.0 MB/s eta 0:00:00


In [6]:
## Understanding output from BertTokenizer 
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

example_text = 'Che cazzo stai dicendo?'
bert_input = tokenizer(example_text,padding='max_length', max_length = 12, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 44131, 11135, 26417, 16527, 10116, 20730, 10605,   136,   102,
             0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [7]:
print(tokenizer.decode(bert_input.input_ids[0]))

[CLS] Che cazzo stai dicendo? [SEP] [PAD] [PAD]


## Defining dataset

In [8]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
labels = {'hateful':0,
          'non-hateful':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label_gold']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['test_case']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


## Train, val and test split in 80:10:10

In [9]:
np.random.seed(112)
df_train, df_val, df_test = np.split(d.sample(frac=1, random_state=42), 
                                     [int(.8*len(d)), int(.9*len(d))])

print(len(df_train),len(df_val), len(df_test))

2952 369 369


## Model Building

In [10]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

## Training

In [11]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1476/1476 [06:06<00:00,  4.02it/s]


Epochs: 1 | Train Loss:  0.261                 | Train Accuracy:  0.751                 | Val Loss:  0.190                 | Val Accuracy:  0.854


100%|██████████| 1476/1476 [06:05<00:00,  4.04it/s]


Epochs: 2 | Train Loss:  0.098                 | Train Accuracy:  0.941                 | Val Loss:  0.078                 | Val Accuracy:  0.951


100%|██████████| 1476/1476 [06:04<00:00,  4.05it/s]


Epochs: 3 | Train Loss:  0.030                 | Train Accuracy:  0.988                 | Val Loss:  0.048                 | Val Accuracy:  0.973


100%|██████████| 1476/1476 [06:04<00:00,  4.04it/s]


Epochs: 4 | Train Loss:  0.017                 | Train Accuracy:  0.994                 | Val Loss:  0.074                 | Val Accuracy:  0.959


100%|██████████| 1476/1476 [06:04<00:00,  4.05it/s]


Epochs: 5 | Train Loss:  0.006                 | Train Accuracy:  0.999                 | Val Loss:  0.045                 | Val Accuracy:  0.976


In [ ]:
## Saving the model
torch.save(model, '/content/drive/MyDrive/')


## Model evaluation

In [59]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    # print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    # print(output, '\n', output.argmax(dim=1))
    return output    
#evaluate(model, df_test)

In [16]:
evaluate(model, df_test)

Test Accuracy:  0.986
tensor([[5.0069, 0.0000]], device='cuda:0') 
 tensor([0], device='cuda:0')


In [19]:
df_test.head()

,mhc_case_id,test_case,label_gold
1603,italian-1604,Dovrebbe essere chiaro che le donne non sono e...,hateful
160,italian-161,Le persone di colore sono assolutamente sprege...,hateful
1704,italian-1705,Non tollererò mai l'idea di ammazzare tutti i ...,non-hateful
956,italian-957,I culattoni non riescono proprio a trattenersi.,hateful
3234,italian-3235,Le persone di colore sono cosìnauseanti.,hateful


In [77]:
## Testing model on a sentence
sentence = "Puoi chiamarmi"

a = evaluate(model, pd.DataFrame({'test_case': [sentence], 'label_gold':['hateful']}))

# hateful = 0
# non hateful = 1

a_list = a.tolist()
if (a_list[0][0]>a_list[0][1]):
  lbl = "hateful"
elif (a_list[0][0]<a_list[0][1]):
  lbl = "non-hateful"

print(lbl)


# Puoi chiamarmi: You can call me: non-hateful
# amo l'India: I love India: non-hateful
# Non mi piace andare al parcheggio: I don't like going to the parking lot: non-hateful
# ti odio: I hate you: hateful
# come osi: How dare you: hateful


non-hateful


In [42]:
type(a)

torch.Tensor

In [49]:
a1 = a.tolist()
print(a1)
print(a1[0])
print(a1[0][0])
print(a1[0][1])

[[0.0, 4.371344566345215]]
[0.0, 4.371344566345215]
0.0
4.371344566345215
